In [1]:
import os
import re
import json
import warnings
import huggingface_hub

from openai import OpenAI

os.environ["TOKENIZERS_PARALLELISM"] = "false"
warnings.filterwarnings("ignore", category=FutureWarning)

from dotenv import load_dotenv
load_dotenv("../keys.env")

upstage_api_key = os.getenv("UPSTAGE_API_KEY")
os.environ['UPSTAGE_API_KEY'] = upstage_api_key

openai_api_key = os.getenv('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = openai_api_key

hf_token = os.getenv("HF_TOKEN")
huggingface_hub.login(hf_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/pervinco/.cache/huggingface/token
Login successful


In [2]:
client = OpenAI()
model = "gpt-4o"

def load_jsonl(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return [json.loads(line) for line in f]
    
def clean_json_response(response):
    # 코드 블록(예: ```json, ```) 제거
    cleaned_response = re.sub(r'```(?:json)?', '', response).strip()
    
    return cleaned_response

In [3]:
def process(dialogue, model, client):
    context = (
    """
    입력된 한국어 문장을 보다 문어체 형식으로 개선해주세요.
    또한 개선된 한국어 문장을 자연스러운 영어 문장으로 개선해주세요.

    반드시 JSON 형식으로 응답하며, 키와 값은 모두 이중 따옴표로 감쌉니다.
    {"ko_query" : "개선된 한국어 문장", "en_query" : "영어로 번역된 문장"}
    
    """
    )
    
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": context},
            {"role": "user", "content": dialogue}
        ],
    )
    
    response = completion.choices[0].message.content
    cleaned_response = clean_json_response(response)
    
    try:
        json_response = json.loads(cleaned_response)
    except json.JSONDecodeError:
        return {"error": "Invalid JSON response", "response": response}
    
    return json_response

In [4]:
def load_query(query_path, model, client, output_path):
    queries = load_jsonl(query_path)
    expanded_queries = []

    for query_data in queries:
        eval_id = query_data['eval_id']
        org_query = query_data['query']['query']
        
        print(eval_id)
        print(org_query)
        
        result = process(org_query, model, client)
        print(result)
        print(result.keys())
        print(result['ko_query'])
        print(result['en_query'], '\n')
        
        expanded_queries.append({
            "eval_id": eval_id,
            "ko_query": result['ko_query'],
            "en_query": result['en_query'],
        })
    
    # Save expanded queries to a jsonl file
    with open(output_path, 'w', encoding='utf-8') as f:
        for query in expanded_queries:
            json.dump(query, f, ensure_ascii=False)
            f.write('\n')

In [5]:
load_query("../dataset/processed_eval.jsonl", model, client, "../dataset/koen_eval.jsonl")

78
나무의 분류에 대해 조사해 보기 위한 방법은?
{'ko_query': '나무의 분류에 대해 조사하는 방법은 무엇인가요?', 'en_query': 'What are the methods for researching the classification of trees?'}
dict_keys(['ko_query', 'en_query'])
나무의 분류에 대해 조사하는 방법은 무엇인가요?
What are the methods for researching the classification of trees? 

213
각 나라에서의 공교육 지출 현황에 대해 알려줘.
{'ko_query': '각 나라의 공교육 지출 현황에 대해 알려주시기 바랍니다.', 'en_query': 'Please provide information on the status of public education expenditure in each country.'}
dict_keys(['ko_query', 'en_query'])
각 나라의 공교육 지출 현황에 대해 알려주시기 바랍니다.
Please provide information on the status of public education expenditure in each country. 

107
기억 상실증의 원인은 무엇인가요?
{'ko_query': '기억 상실증의 원인은 무엇인가요?', 'en_query': 'What are the causes of amnesia?'}
dict_keys(['ko_query', 'en_query'])
기억 상실증의 원인은 무엇인가요?
What are the causes of amnesia? 

81
통학 버스의 가치에 대해 말해줘.
{'ko_query': '통학 버스의 중요성에 대해 설명해 주세요.', 'en_query': 'Please explain the importance of school buses.'}
dict_keys(['ko_query', 'en_query'])
통학 버스의 중요성에 대해 설명해